## Test out the neural network algorithm

Use the mnist dataset to test out the neural network algo different option. Remove all classes except for 0 and 1 to turn this very simple binary classification problem.

In [18]:
import numpy as np

In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
from NeuralNetNLayer import NeuralNet

**Load in and standardize the dataset**

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
np.random.seed(1)

In [7]:
df = pd.read_csv('mnist/train.csv')

In [8]:
# Keep only the first two numbers (0 and 1) to convert this to binary classification
df_two = df[df['label'] <= 1]
df_two.label.value_counts()

1    4684
0    4132
Name: label, dtype: int64

In [95]:
# Check out the data
df_two.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Split into X and y training and validation sets 
X_train, X_test, y_train, y_test = train_test_split(df_two.drop('label', axis=1), df_two['label'], train_size = 0.67)

In [12]:
# Standardize Before transposing (skelearn assumes obs x features format)
scaler = StandardScaler()
scaler = scaler.fit(X_train)
X_s = scaler.transform(X_train)
# Transpose X and convert to an array for compatibility with the neural network
# neural network requires features as rows and observations as columns
X_t = X_s.T
X_t.shape

(784, 5906)

In [13]:
# Transpose y and convert to an array
y_t = y_train.values.reshape(1, len(y_train))
y_t.shape

(1, 5906)

**Create and train the test models**

In [94]:
from NeuralNetNLayer import NeuralNet

In [88]:
# Create and train a base model with random initialization
clf = NeuralNet(X_t, y_t, [6, 3, 1], initialization = 'he', rseed=9)

In [89]:
clf.train(0.3, 500)

batch size:  5906
Iteration: 0 , Cost, Accuracy, lr:  0.6784773688887491 92.32983406705046 0.3
Iteration: 100 , Cost, Accuracy, lr:  0.0026876292734848515 99.96613613274636 0.3
Iteration: 200 , Cost, Accuracy, lr:  0.0009797340178566976 100.0 0.3
Iteration: 300 , Cost, Accuracy, lr:  0.0005832651822632082 100.0 0.3
Iteration: 400 , Cost, Accuracy, lr:  0.00040211886679553376 100.0 0.3


In [86]:
# Create and train the a deeper model with regularization
clf_reg = NeuralNet(X_t, y_t, [6, 4, 4, 1], initialization='he', rseed=9)

In [87]:
clf_reg.train(0.5, 500, lambd=0.2)

batch size:  5906
Iteration: 0 , Cost, Accuracy, lr:  0.6458077453967446 88.36776159837454 0.5
Iteration: 100 , Cost, Accuracy, lr:  0.023709312532213242 99.98306806637318 0.5
Iteration: 200 , Cost, Accuracy, lr:  0.011827699290228581 99.98306806637318 0.5
Iteration: 300 , Cost, Accuracy, lr:  0.00782447881528291 100.0 0.5
Iteration: 400 , Cost, Accuracy, lr:  0.005937730463556822 100.0 0.5


In [43]:
# Test training with minibatch and momentum
clf_mb = NeuralNet(X_t, y_t, [6, 4, 4, 1], initialization='he', rseed=9)

In [44]:
clf_mb.train(0.5, 500, lambd = 0.2, batch_size=512, beta=0.9)

batch size:  512
Iteration: 0 , Cost, Accuracy 0.2790536231403001 99.37351845580766
Iteration: 100 , Cost, Accuracy 0.009524371825295563 100.0
Iteration: 200 , Cost, Accuracy 0.007050646153984092 100.0
Iteration: 300 , Cost, Accuracy 0.005623237898762064 100.0
Iteration: 400 , Cost, Accuracy 0.004771801005267266 100.0


In [82]:
# add in learning rate decay
clf_decay = NeuralNet(X_t, y_t, [6, 4, 4, 1], initialization='he', rseed=9)

In [83]:
clf_decay.train(1.0, 500, lambd = 0.2, batch_size=512, beta=0.9, decay_rate=0.3)

batch size:  512
Iteration: 0 , Cost, Accuracy, lr:  0.22057206159675233 99.55976972570268 1.0
Iteration: 100 , Cost, Accuracy, lr:  0.007271494229365515 100.0 1.0
Iteration: 200 , Cost, Accuracy, lr:  0.005182296670023927 100.0 0.7692307692307692
Iteration: 300 , Cost, Accuracy, lr:  0.004377130083190326 100.0 0.625
Iteration: 400 , Cost, Accuracy, lr:  0.003990802480662113 100.0 0.5263157894736842


**Test accuracy on the validation dataset**

In [37]:
# Standardize (using the original values) and transform
X_s = scaler.transform(X_test)
X_v = X_s.T
X_v.shape

(784, 2910)

In [38]:
y_v = y_test.values.reshape(1, len(y_test))
y_v.shape

(1, 2910)

In [90]:
clf.validation_accuracy(X_v=X_v, y_v=y_v)

99.7594501718213

In [91]:
clf_reg.validation_accuracy(X_v=X_v, y_v=y_v)

99.65635738831615

In [92]:
clf_mb.validation_accuracy(X_v=X_v, y_v=y_v)

99.93127147766323

In [93]:
clf_decay.validation_accuracy(X_v=X_v, y_v=y_v)

99.96563573883162

## Algorithm and Math Details

### Algorithm Steps

**Forward Propagation**

**Backpropagation**

First initalize the last layer using the derivative of the sigmoid (for 0/1 classification)

$dZ = A^{[L]} - y $

$dW = \frac{1}{m}*dZ • A^{[L-1]}$

$dB = \frac{1}{m}*\sum_{i} dZ $

Next calculate dA[L-1] to use in the next layer's dZ

$ dA^{[L-1]} = W.T•\frac{dJ}{dZ}  $

### Calculus Derivations

cost function, from:
$$J = -\frac{1}{m} \sum\limits_{i = 1}^{m} \large{(}\small  y^{(i)}\log\left(a^{[L](i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right) \large{)} \tag{1}$$

**Last Layer L**



$$\frac{dJ}{dW^{[L]}} =  \frac{dJ}{dA^{[L]}} \frac{dA^{[L]}}{dW^{[L]}} $$

$$ = \frac{dJ}{dA^{[L]}} \frac{dA^{[L]}}{dZ^{[L]}} \frac{dZ^{[L]}}{dW^{[L]}}$$

$$ \frac{dJ}{dA^{[L]}} = \frac{y}{A^{[L]}} - \frac{1-y}{1-A^{[L]}}$$

$$ A^{[L]} = \sigma(Z^{[L]})   $$  


$$ \frac{dA^{[L]}}{dZ^{[L]}} =  \sigma(Z^{[L]})(1-\sigma(Z^{[L]})) = A^{[L]}(1-A^{[L]}) $$

$$ \frac{dJ}{dZ^{[L]}} = {dZ^{[L]}} =  \frac{dJ}{dA^{[L]}} \frac{dA^{[L]}}{dZ^{[L]}} = \frac{A^{[L]}(1-A^{[L]})y}{A^{[L]}} - \frac{A^{[L]}(1-A^{[L]})(1-y)}{1-A^{[L]}}  $$

$$ = y - A^{[L]}y - A^{[L]} + A^{[L]}y
= y - A^{[L]} $$

$$ dZ^{[L]} = y - A^{[L]} $$

$$ Z^{[L]} = W^{[L]}.T•A^{[L-1]} + b^{[L]} $$

$$ \frac{dZ^{[L]}}{dW^{[L]}} = A^{[L-1]}$$

$$ dW^{[L]} = (y - A^{[L]})A^{[L-1]} $$

**Backpropagation Layers**

$$\frac{dJ}{dW^{[l]}} =  \frac{1}{m}\frac{dJ}{dA^{[l]}} \frac{dA^{[l]}}{dZ^{[l]}} \frac{dZ^{[l]}}{dW^{[l]}}$$ 

In the previous step, calculate the next layers dA using dZ from the previous layer

$$\frac{dJ}{dA^{[l]}} = \frac{dJ}{dZ^{[l+1]}}\frac{dZ^{[l+1]}}{dA^{[l]}} $$

A in terms of Al-1
$$ A^{[l+1]} = g(W^{[l+1]T}A^{[l]})  $$

Z_l is a function of A_l-1
$$ Z^{[l+1]} = W^{[l+1]T}A^{[l]} + b^{[l+1]} $$

$$ \frac{dZ^{[l+1]}}{dA^{[l]}} = W^{[l+1]T} $$

So the layer's calculations can easily give us dAl

$$\frac{dJ}{dA^{[l]}} = W^{[l+1]T} • \frac{dJ}{dZ^{[l+1]}} $$

Now use the dA_l-1 in the next layer (calclute the rest of the things)

$$ \frac{dA^{[l]}}{dZ^{[l]}} = g'(Z^{[l]})$$

$$ \frac{dZ^{[l]}}{dW^{[l]}} = W^{[l]} $$

so putting it all togther

$$\frac{dJ}{dW^{[l]}} =  \frac{1}{m}\frac{dJ}{dA^{[l]}} \frac{dA^{[l]}}{dZ^{[l]}} \frac{dZ^{[l]}}{dW^{[l]}} = \frac{1}{m}\frac{dJ}{dA^{[l]}} g'(Z^{[l]})  W^{[l]}
$$ 

Where dAl was calculated in the previous layer already

Save the dZ first part of this for use in the next layer:

$$ dZ^{[l]} = \frac{dJ}{dA^{[l]}} \frac{dA^{[l]}}{dZ^{[l]}} = 
\frac{dJ}{dA^{[l]}} g'(Z^{[l]}) $$

- Train on a single example (use L instead of J) is how the math works most easily
- Applying the gradient from each sample separately is stochastic gradient descent
- Applying all at once is regular gradient descent - extend by taking the average gradient (1/m)*grad
- Mini batch is somewhere in between

$$ \frac{dJ}{dA^{[L]}} = \frac{y}{\sigma(Z^{[L]})} - \frac{1-y}{1-\sigma(Z^{[L]})}$$

In [1]:
print i


SyntaxError: Missing parentheses in call to 'print'. Did you mean print(i)? (908402389.py, line 1)

In [2]:
i = 5
print(i)

5
